In [1]:
%load_ext autoreload
%autoreload 2
#tensorboard doesn't work with keras it seems, needs tf.keras
from preprocess import *
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Convolution1D, GlobalAveragePooling1D, GlobalMaxPool1D, Dense, Dropout,
                                     Flatten, Conv2D, MaxPooling2D, Input, MaxPool1D, concatenate)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras import losses, models, optimizers
from tensorflow.keras.callbacks import TensorBoard
import time  
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
labels = get_labels('./data')  
num_classes = len(labels[0])
labels

(['shh', 'silence', 'ttt'], array([0, 1, 2]), array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], dtype=float32))

In [3]:
#len of longest .wav data
max_sample_length = max_sample_len()
max_sample_length

16384

In [4]:
save_raw_data_to_array()

Saving vectors of label - 'ttt': 100%|██████████| 229/229 [00:00<00:00, 4660.77it/s]


In [5]:
# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
len(X_train)

(421, 16384)
(421,)
(282, 16384)
(282,)


421

In [7]:
X_train[:20]

array([[-2.4414062e-04, -2.1362305e-04, -1.2207031e-04, ...,
         6.1035156e-05,  3.0517578e-05,  3.0517578e-05],
       [-3.0517578e-04, -2.7465820e-04, -2.1362305e-04, ...,
        -5.7983398e-04, -7.3242188e-04, -8.2397461e-04],
       [ 3.0517578e-04,  3.0517578e-04,  2.7465820e-04, ...,
         0.0000000e+00,  0.0000000e+00,  3.0517578e-05],
       ...,
       [-6.1035156e-05, -1.2207031e-04, -1.5258789e-04, ...,
         9.1552734e-05,  6.1035156e-05,  3.0517578e-05],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-2.7465820e-04, -2.1362305e-04, -1.8310547e-04, ...,
         2.4414062e-04,  3.0517578e-04,  3.0517578e-04]], dtype=float32)

In [8]:
#ie [1,2,3] becomes [[1],[2],[3]]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

y_test_hot[:2]

array([[0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [9]:
X_train[:20]

array([[[-2.4414062e-04],
        [-2.1362305e-04],
        [-1.2207031e-04],
        ...,
        [ 6.1035156e-05],
        [ 3.0517578e-05],
        [ 3.0517578e-05]],

       [[-3.0517578e-04],
        [-2.7465820e-04],
        [-2.1362305e-04],
        ...,
        [-5.7983398e-04],
        [-7.3242188e-04],
        [-8.2397461e-04]],

       [[ 3.0517578e-04],
        [ 3.0517578e-04],
        [ 2.7465820e-04],
        ...,
        [ 0.0000000e+00],
        [ 0.0000000e+00],
        [ 3.0517578e-05]],

       ...,

       [[-6.1035156e-05],
        [-1.2207031e-04],
        [-1.5258789e-04],
        ...,
        [ 9.1552734e-05],
        [ 6.1035156e-05],
        [ 3.0517578e-05]],

       [[ 0.0000000e+00],
        [ 0.0000000e+00],
        [ 0.0000000e+00],
        ...,
        [ 0.0000000e+00],
        [ 0.0000000e+00],
        [ 0.0000000e+00]],

       [[-2.7465820e-04],
        [-2.1362305e-04],
        [-1.8310547e-04],
        ...,
        [ 2.4414062e-04],
        [ 3.051

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train_hot.shape)#2nd dim is the number of labels

(421, 16384, 1)
(421,)
(282, 16384, 1)
(282,)
(421, 3)


In [11]:
max_data = np.max(X_train)
min_data = np.min(X_train)
def audio_norm(data, max_data, min_data):
    data = (data-min_data)/(max_data-min_data+1e-6)
    return data-0.5

In [12]:
X_test[:20]

array([[[-1.8310547e-04],
        [-1.8310547e-04],
        [-2.4414062e-04],
        ...,
        [-3.6621094e-04],
        [-3.3569336e-04],
        [-3.6621094e-04]],

       [[-1.5258789e-04],
        [-2.4414062e-04],
        [-2.4414062e-04],
        ...,
        [ 9.1552734e-05],
        [ 6.1035156e-05],
        [ 6.1035156e-05]],

       [[ 0.0000000e+00],
        [ 0.0000000e+00],
        [ 0.0000000e+00],
        ...,
        [ 0.0000000e+00],
        [ 0.0000000e+00],
        [ 0.0000000e+00]],

       ...,

       [[ 1.8310547e-04],
        [-6.1035156e-05],
        [-2.1362305e-04],
        ...,
        [ 7.9345703e-04],
        [ 4.5776367e-04],
        [ 6.4086914e-04]],

       [[ 6.1035156e-05],
        [ 9.1552734e-05],
        [ 9.1552734e-05],
        ...,
        [-6.1035156e-05],
        [-6.1035156e-05],
        [-1.5258789e-04]],

       [[ 0.0000000e+00],
        [ 0.0000000e+00],
        [ 0.0000000e+00],
        ...,
        [-2.4414062e-04],
        [-2.136

In [13]:
X_train = audio_norm(X_train, max_data, min_data)
X_test = audio_norm(X_test, max_data, min_data)
X_test[:20]

array([[[0.01105839],
        [0.01105839],
        [0.01095694],
        ...,
        [0.01075399],
        [0.01080471],
        [0.01075399]],

       [[0.01110911],
        [0.01095694],
        [0.01095694],
        ...,
        [0.01151496],
        [0.01146424],
        [0.01146424]],

       [[0.01136279],
        [0.01136279],
        [0.01136279],
        ...,
        [0.01136279],
        [0.01136279],
        [0.01136279]],

       ...,

       [[0.01166713],
        [0.01126128],
        [0.01100767],
        ...,
        [0.01268178],
        [0.0121237 ],
        [0.0124281 ]],

       [[0.01146424],
        [0.01151496],
        [0.01151496],
        ...,
        [0.01126128],
        [0.01126128],
        [0.01110911]],

       [[0.01136279],
        [0.01136279],
        [0.01136279],
        ...,
        [0.01095694],
        [0.01100767],
        [0.01100767]]], dtype=float32)

In [14]:
def get_1d_conv_model(input_length):
    
    
    inp = Input(shape=(input_length,1))
   
    x = Convolution1D(16, 9, activation=relu, padding="valid")(inp)
    x = Convolution1D(16, 9, activation=relu, padding="valid")(x)
    x = MaxPool1D(16)(x)
    x = Dropout(rate=0.1)(x)
    
    x = Convolution1D(32, 3, activation=relu, padding="valid")(x)
    x = Convolution1D(32, 3, activation=relu, padding="valid")(x)
    x = MaxPool1D(4)(x)
    x = Dropout(rate=0.1)(x)
    
    x = Convolution1D(128, 3, activation=relu, padding="valid")(x)
    x = Convolution1D(128, 3, activation=relu, padding="valid")(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(rate=0.2)(x)

    x = Dense(32, activation=relu)(x)
    x = Dense(256, activation=relu)(x)
    out = Dense(num_classes, activation=softmax)(x)

    model = models.Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])
    return model

# Building The Model Then Training it

In [15]:
NAME = "1dconv{}".format(int(time.time()))
tensorboard_cb = TensorBoard(log_dir="logs/{}".format(NAME), histogram_freq=1)
%load_ext tensorboard
tensorboard_cb

In [16]:
model = get_1d_conv_model(X_train.shape[1])
model.fit(X_train, y_train_hot, batch_size=100, epochs=17, verbose=True, validation_data=(X_test, y_test_hot), callbacks=[tensorboard_cb])

Train on 421 samples, validate on 282 samples
Epoch 1/17
421/421 [==============================] - 5s 12ms/sample - loss: 1.0955 - acc: 0.3325 - val_loss: 1.0801 - val_acc: 0.4574
Epoch 2/17
421/421 [==============================] - 4s 10ms/sample - loss: 1.0651 - acc: 0.3634 - val_loss: 0.9918 - val_acc: 0.4433
Epoch 3/17
421/421 [==============================] - 4s 10ms/sample - loss: 0.9552 - acc: 0.3492 - val_loss: 0.7859 - val_acc: 0.4681
Epoch 4/17
421/421 [==============================] - 4s 10ms/sample - loss: 0.8326 - acc: 0.5914 - val_loss: 0.7064 - val_acc: 0.7518
Epoch 5/17
421/421 [==============================] - 4s 10ms/sample - loss: 0.7692 - acc: 0.7055 - val_loss: 0.6494 - val_acc: 0.8050
Epoch 6/17
421/421 [==============================] - 4s 10ms/sample - loss: 0.7405 - acc: 0.6556 - val_loss: 0.6557 - val_acc: 0.6702
Epoch 7/17
421/421 [==============================] - 4s 11ms/sample - loss: 0.7230 - acc: 0.6817 - val_loss: 0.6172 - val_acc: 0.6773
Epoch 8/1

In [ ]:
%tensorboard --logdir logs/ --bind_all

In [19]:
save_stats(NAME, max_data=max_data, min_data=min_data)
save_model(model, NAME)
NAME

'1dconv1584402278'

## Prediction

In [18]:
def predict(filepath, model):
    wave, sr = librosa.load(filepath, mono=True, sr=16000)
    wave = random_pad(wave, max_sample_length)
    wave = audio_norm(wave, max_data, min_data)
    wave = wave.reshape(1, wave.shape[0], 1)
    return get_labels()[0][
            np.argmax(model.predict(wave))
    ]
print(predict('./data/shh/W9M3W3YR2.wav', model=model))

shh
